# Run some constrained projections

Follow the same setup as in notebook 200, with more scrutiny of outputs including other forcers.

We need to save out the land use forcing in each SSP and ensemble member to feed into the concentration-driven runs (240).

## Basic imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
from tqdm import tqdm
import xarray as xr

from fair import FAIR
from fair.io import read_properties
from fair.interface import fill, initialise
from fair.forcing.ghg import meinshausen2020

## Set up problem

In [ ]:
erf_2co2 = meinshausen2020(
    np.array([554.30, 731.41, 273.87]) * np.ones((1, 1, 1, 3)),
    np.array([277.15, 731.41, 273.87]) * np.ones((1, 1, 1, 3)),
    np.array((1.05, 0.86, 1.07)) * np.ones((1, 1, 1, 1)),
    np.ones((1, 1, 1, 3)),
    np.array([True, False, False]),
    np.array([False, True, False]),
    np.array([False, False, True]),
    np.array([False, False, False])
).squeeze()[0]
erf_2co2

In [ ]:
scenarios = ['rcp26', 'rcp45', 'rcp60', 'rcp85']

In [ ]:
df_forc = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')
df_solar = pd.read_csv('../data/forcing/solar_erf.csv', index_col='year')

In [ ]:
solar_forcing = np.zeros(536)
solar_forcing[:535] = df_solar.loc[1765.5:, 'solar_erf'].values
volcanic_forcing = np.zeros(536)
volcanic_forcing[:255] = df_forc['volcanic'].values[15:]
volcanic_forcing[254:266] = np.linspace(1, 0, 12) * volcanic_forcing[254]

In [ ]:
pl.plot(solar_forcing)

In [ ]:
da_emissions = xr.load_dataarray('../data/rcmip/rcp_emissions_fair2.1.nc')

In [ ]:
valid_all = np.loadtxt('../data/ar6_ensemble_batches_rcp/final_ebm3.csv').astype(np.int64)#[:1000]
valid_all

In [ ]:
df_cc=pd.read_csv('../data/parameter_sets/carbon_cycle.csv')
df_cr=pd.read_csv('../data/parameter_sets/climate_response.csv')
df_aci=pd.read_csv('../data/parameter_sets/erfaci.csv')
df_ari=pd.read_csv('../data/parameter_sets/erfari.csv')
df_ozone=pd.read_csv('../data/parameter_sets/ozone.csv')
df_scaling=pd.read_csv('../data/parameter_sets/forcing_scaling.csv')
df_1750co2=pd.read_csv('../data/parameter_sets/co2_concentration_1750.csv')

In [ ]:
species = [
    'CO2 FFI', 'CO2 AFOLU', 'CO2', 'CH4', 'N2O',
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1202', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C6F14',
    'SF6',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-227ea', 'HFC-23', 'HFC-245fa', 'HFC-32', 
    'HFC-4310mee', 'Solar', 'Volcanic', 'Aerosol-radiation interactions',
    'Aerosol-cloud interactions', 'Ozone', 'Light absorbing particles on snow and ice',
    'Land use', 'Stratospheric water vapour', 'Equivalent effective stratospheric chlorine',
    'Contrails', 'NOx aviation'
]

species, properties = read_properties(species=species)

In [ ]:
seed = 1355763 + 399 * valid_all

In [ ]:
trend_shape = np.ones(536)
trend_shape[:256] = np.linspace(0, 1, 256)

In [ ]:
f = FAIR(ch4_method='Thornhill2021')
f.define_time(1765, 2300, 1)
f.define_scenarios(scenarios)
f.define_configs(valid_all)
f.define_species(species, properties)
f.allocate()

In [ ]:
n_configs = len(valid_all)
n_scenarios = len(scenarios)

In [ ]:
da = da_emissions.loc[dict(config='unspecified')][:535, ...]
fe = da.expand_dims(dim='config', axis=2)
f.emissions = fe.drop('config') * np.ones((1, 1, n_configs, 1))
f.emissions[:86,0,:,5] = np.linspace(2, f.emissions[85,0,0,5], 86)[:, None]
f.emissions[:86,0,:,6] = np.linspace(1.2, f.emissions[85,0,0,6], 86)[:, None]
f.emissions[:86,0,:,7] = np.linspace(10, f.emissions[85,0,0,7], 86)[:, None]
f.emissions[:86,0,:,8] = np.linspace(4, f.emissions[85,0,0,8], 86)[:, None]
f.emissions[:86,0,:,9] = np.linspace(46/14*2, f.emissions[85,0,0,9], 86)[:, None]
f.emissions[:86,0,:,10] = np.linspace(10, f.emissions[85,0,0,10], 86)[:, None]
f.emissions[:86,0,:,11] = np.linspace(174, f.emissions[85,0,0,11], 86)[:, None]

fill(f.forcing, volcanic_forcing[:, None, None] * df_scaling.loc[valid_all, 'Volcanic'].values.squeeze(), specie='Volcanic')
fill(f.forcing, 
     solar_forcing[:, None, None] * 
     df_scaling.loc[valid_all, 'solar_amplitude'].values.squeeze() + 
     trend_shape[:, None, None] * df_scaling.loc[valid_all, 'solar_trend'].values.squeeze(),
     specie='Solar'
)

In [ ]:
# using an extremely crude curve fit to estimate 1765 CO2

obs_co2 = np.array([278.3, 285.5, 296.4])
obs_years = [1750, 1850, 1900]

def fit(x, a, b, c):
    return a + b*np.exp(c*(x-1750))

p, _ = scipy.optimize.curve_fit(fit, obs_years, obs_co2, p0=[278.3, 1.45, 0.0171])
pl.plot(np.arange(1750, 1900), p[0]+p[1]*np.exp(p[2] * (np.arange(1750, 1900)-1750)))
pl.scatter(obs_years, obs_co2, color='k')

offset1765co2 = fit(1765, p[0], p[1], p[2]) - fit(1750, p[0], p[1], p[2])
offset1765co2

In [ ]:
df_1765co2 = df_1750co2 + offset1765co2

In [ ]:
obs_ch4 = np.array([729.2, 807.6, 925.1])
obs_years = [1750, 1850, 1900]

def fit(x, a, b, c):
    return a + b*np.exp(c*(x-1750))

p, _ = scipy.optimize.curve_fit(fit, obs_years, obs_ch4, p0=[278.3, 1.45, 0.0171])
pl.plot(np.arange(1750, 1900), p[0]+p[1]*np.exp(p[2] * (np.arange(1750, 1900)-1750)))
pl.scatter(obs_years, obs_ch4, color='k')

baseline1765ch4 = fit(1765, p[0], p[1], p[2])
baseline1765ch4

In [ ]:
obs_n2o = np.array([270.1, 272.1, 278.9])
obs_years = [1750, 1850, 1900]

def fit(x, a, b, c):
    return a + b*np.exp(c*(x-1750))

p, _ = scipy.optimize.curve_fit(fit, obs_years, obs_n2o, p0=[278.3, 1.45, 0.0171])
pl.plot(np.arange(1750, 1900), p[0]+p[1]*np.exp(p[2] * (np.arange(1750, 1900)-1750)))
pl.scatter(obs_years, obs_n2o, color='k')

baseline1765n2o = fit(1765, p[0], p[1], p[2])
baseline1765n2o

In [ ]:
calibrated_f4co2_mean = df_cr['F_4xCO2'].mean()

# climate response
fill(f.climate_configs['ocean_heat_capacity'], df_cr.loc[valid_all, 'c1':'c3'].values)
fill(f.climate_configs['ocean_heat_transfer'], df_cr.loc[valid_all, 'kappa1':'kappa3'].values)
fill(f.climate_configs['deep_ocean_efficacy'], df_cr.loc[valid_all, 'epsilon'].values.squeeze())
fill(f.climate_configs['gamma_autocorrelation'], df_cr.loc[valid_all, 'gamma'].values.squeeze())
fill(f.climate_configs['sigma_eta'], df_cr.loc[valid_all, 'sigma_eta'].values.squeeze())
fill(f.climate_configs['sigma_xi'], df_cr.loc[valid_all, 'sigma_xi'].values.squeeze())
fill(f.climate_configs['seed'], seed)
fill(f.climate_configs['stochastic_run'], True)
fill(f.climate_configs['use_seed'], True)
fill(f.climate_configs['forcing_4co2'], 2 * erf_2co2 * (1 + 0.561*(calibrated_f4co2_mean - df_cr.loc[valid_all,'F_4xCO2'])/calibrated_f4co2_mean))

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs['iirf_0'], df_cc.loc[valid_all, 'r0'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_airborne'], df_cc.loc[valid_all, 'rA'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_uptake'], df_cc.loc[valid_all, 'rU'].values.squeeze(), specie='CO2')
fill(f.species_configs['iirf_temperature'], df_cc.loc[valid_all, 'rT'].values.squeeze(), specie='CO2')

# aerosol indirect
fill(f.species_configs['aci_scale'], df_aci.loc[valid_all, 'beta'].values.squeeze())
fill(f.species_configs['aci_shape'], df_aci.loc[valid_all, 'shape_so2'].values.squeeze(), specie='Sulfur')
fill(f.species_configs['aci_shape'], df_aci.loc[valid_all, 'shape_bc'].values.squeeze(), specie='BC')
fill(f.species_configs['aci_shape'], df_aci.loc[valid_all, 'shape_oc'].values.squeeze(), specie='OC')

# methane lifetime baseline
fill(f.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people might wanna run pulse expts with these gases)
fill(f.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

# aerosol direct
for specie in df_ari:
    fill(f.species_configs['erfari_radiative_efficiency'], df_ari.loc[valid_all, specie], specie=specie)

# forcing
for specie in df_scaling:
    if specie in ['minorGHG', 'solar_amplitude', 'solar_trend', 'CO2', 'Volcanic']:
        continue
    fill(f.species_configs['forcing_scale'], df_scaling.loc[valid_all, specie].values.squeeze(), specie=specie)
for specie in ['CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115', 'HCFC-22', 'HCFC-141b', 'HCFC-142b',
        'CCl4', 'CH3Cl', 'CH3CCl3', 'CH3Br', 'Halon-1202', 'Halon-1211', 'Halon-1301', 'Halon-2402',
        'CF4', 'C2F6', 'C6F14', 'SF6',
        'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-227ea', 'HFC-23', 'HFC-245fa', 'HFC-32', 'HFC-4310mee']:
    fill(f.species_configs['forcing_scale'], df_scaling.loc[valid_all, 'minorGHG'].values.squeeze(), specie=specie)
fill(f.species_configs['forcing_scale'], 1 + 0.561*(calibrated_f4co2_mean - df_cr.loc[valid_all,'F_4xCO2'].values)/calibrated_f4co2_mean, specie='CO2')

# ozone
for specie in df_ozone:
    fill(f.species_configs['ozone_radiative_efficiency'], df_ozone.loc[valid_all, specie], specie=specie)

# tune down volcanic efficacy
fill(f.species_configs['forcing_efficacy'], 0.6, specie='Volcanic')


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs['baseline_concentration'], df_1765co2.loc[valid_all, 'co2_concentration'].values.squeeze(), specie='CO2')

# initial condition of other species
fill(f.species_configs['baseline_concentration'], baseline1765ch4, specie='CH4')
fill(f.species_configs['baseline_concentration'], baseline1765n2o, specie='N2O')
fill(f.species_configs['baseline_emissions'], 2, specie='Sulfur')
fill(f.species_configs['baseline_emissions'], 174, specie='CO')
fill(f.species_configs['baseline_emissions'], 10, specie='VOC')
fill(f.species_configs['baseline_emissions'], 4, specie='NH3')
fill(f.species_configs['baseline_emissions'], 2*46/14, specie='NOx')
fill(f.species_configs['baseline_emissions'], 1.2, specie='BC')
fill(f.species_configs['baseline_emissions'], 10, specie='OC')

# initial conditions
initialise(f.concentration, f.species_configs['baseline_concentration'])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

f.run()

In [ ]:
f.species_configs

In [ ]:
fig, ax = pl.subplots(1, 4, figsize=(16, 5))

for i in range(4):
    ax[i].fill_between(
        f.timebounds, 
        np.min(f.temperature[:, i, :, 0]-f.temperature[85:136, i, :, 0].mean(axis=0), axis=1), 
        np.max(f.temperature[:, i, :, 0]-f.temperature[85:136, i, :, 0].mean(axis=0), axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i].fill_between(
        f.timebounds, 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[85:136, i, :, 0].mean(axis=0), 5, axis=1), 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[85:136, i, :, 0].mean(axis=0), 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i].fill_between(
        f.timebounds, 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[85:136, i, :, 0].mean(axis=0), 16, axis=1), 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[85:136, i, :, 0].mean(axis=0), 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i].plot(
        f.timebounds, 
        np.median(f.temperature[:, i, :, 0]-f.temperature[85:136, i, :, 0].mean(axis=0), axis=1), 
        color='#000000',
    )
    ax[i].set_xlim(1750,2300)
    ax[i].set_ylim(-1, 10)
    ax[i].axhline(0, color='k', ls=":", lw=0.5)
    ax[i].set_title(scenarios[i])
pl.suptitle('Temperature anomaly')

# Temperature diffs w.r.t. 1995-2014

Future periods are 2021-2040, 2041-2060, 2081-2100. Values are 5th, 50th, 95th percentile.

In [ ]:
# rcp26
print(np.percentile(f.temperature[256:276, 0, :, 0].mean(axis=0)-f.temperature[230:250, 0, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[276:296, 0, :, 0].mean(axis=0)-f.temperature[230:250, 0, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[316:336, 0, :, 0].mean(axis=0)-f.temperature[230:250, 0, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# rcp45
print(np.percentile(f.temperature[256:276, 1, :, 0].mean(axis=0)-f.temperature[230:250, 1, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[276:296, 1, :, 0].mean(axis=0)-f.temperature[230:250, 1, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[316:336, 1, :, 0].mean(axis=0)-f.temperature[230:250, 1, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# rcp60
print(np.percentile(f.temperature[256:276, 2, :, 0].mean(axis=0)-f.temperature[230:250, 2, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[276:296, 2, :, 0].mean(axis=0)-f.temperature[230:250, 2, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[316:336, 2, :, 0].mean(axis=0)-f.temperature[230:250, 2, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# rcp85
print(np.percentile(f.temperature[271:291, 3, :, 0].mean(axis=0)-f.temperature[230:250, 3, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[276:296, 3, :, 0].mean(axis=0)-f.temperature[230:250, 3, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[316:336, 3, :, 0].mean(axis=0)-f.temperature[230:250, 3, :, 0].mean(axis=0), (5, 50, 95)))

# Temperature diffs w.r.t. 1850-1900

In [ ]:
# rcp26
print(np.percentile(f.temperature[256:276, 0, :, 0].mean(axis=0)-f.temperature[85:136, 0, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[276:296, 0, :, 0].mean(axis=0)-f.temperature[85:136, 0, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[316:336, 0, :, 0].mean(axis=0)-f.temperature[85:136, 0, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# rcp45
print(np.percentile(f.temperature[256:276, 1, :, 0].mean(axis=0)-f.temperature[85:136, 1, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[276:296, 1, :, 0].mean(axis=0)-f.temperature[85:136, 1, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[316:336, 1, :, 0].mean(axis=0)-f.temperature[85:136, 1, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# rcp60
print(np.percentile(f.temperature[256:276, 2, :, 0].mean(axis=0)-f.temperature[85:136, 2, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[276:296, 2, :, 0].mean(axis=0)-f.temperature[85:136, 2, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[316:336, 2, :, 0].mean(axis=0)-f.temperature[85:136, 2, :, 0].mean(axis=0), (5, 50, 95)))

In [ ]:
# rcp85
print(np.percentile(f.temperature[271:291, 3, :, 0].mean(axis=0)-f.temperature[85:136, 3, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[276:296, 3, :, 0].mean(axis=0)-f.temperature[85:136, 3, :, 0].mean(axis=0), (5, 50, 95)))
print(np.percentile(f.temperature[316:336, 3, :, 0].mean(axis=0)-f.temperature[85:136, 3, :, 0].mean(axis=0), (5, 50, 95)))

## methane concs

In [ ]:
fig, ax = pl.subplots(1, 4, figsize=(16, 5))

for i in range(4):
    ax[i].fill_between(
        f.timebounds, 
        np.min(f.concentration[:, i, :, 3], axis=1), 
        np.max(f.concentration[:, i, :, 3], axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i].fill_between(
        f.timebounds, 
        np.percentile(f.concentration[:, i, :, 3], 5, axis=1), 
        np.percentile(f.concentration[:, i, :, 3], 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i].fill_between(
        f.timebounds, 
        np.percentile(f.concentration[:, i, :, 3], 16, axis=1), 
        np.percentile(f.concentration[:, i, :, 3], 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i].plot(
        f.timebounds, 
        np.median(f.concentration[:, i, :, 3], axis=1), 
        color='#000000',
    )
    ax[i].set_xlim(1750,2300)
    ax[i].set_ylim(0, 6000)
    ax[i].set_title(scenarios[i])
pl.suptitle('Methane concentration')

## Today

In [ ]:
print(np.percentile(f.temperature[230:250, 1, :, 0].mean(axis=0)-f.temperature[85:136, 1, :, 0].mean(axis=0), (5, 50, 95)))

## Save out parameters

In [ ]:
params_out = pd.concat((df_cr.loc[valid_all, :], df_cc.loc[valid_all, :], df_ari.loc[valid_all, :], df_aci.loc[valid_all, :], df_ozone.loc[valid_all, :], df_scaling.loc[valid_all, :], df_1750co2.loc[valid_all, :]), axis=1)

In [ ]:
params_out.columns

In [ ]:
params_out.columns = ['gamma', 'c1', 'c2', 'c3', 'kappa1', 'kappa2', 'kappa3', 'epsilon',
       'sigma_eta', 'sigma_xi', 'F_4xCO2', 'r0', 'rU', 'rT', 'rA', 'ari BC', 'ari CH4',
       'ari CO', 'ari N2O', 'ari NH3', 'ari NOx', 'ari OC', 'ari Sulfur', 'ari VOC',
       'ari Equivalent effective stratospheric chlorine', 'shape_so2',
       'shape_bc', 'shape_oc', 'beta', 'o3 CH4', 'o3 N2O',
       'o3 Equivalent effective stratospheric chlorine', 'o3 CO', 'o3 VOC', 'o3 NOx',
       'scale CO2', 'scale CH4', 'scale N2O', 'scale minorGHG', 'scale Stratospheric water vapour',
       'scale Contrails', 'scale Light absorbing particles on snow and ice', 'scale Land use',
       'scale Volcanic', 'solar_amplitude', 'solar_trend', 'co2_concentration_1750']

In [ ]:
params_out.drop(columns=['scale CO2', 'ari CO'], inplace=True)

In [ ]:
params_out.to_csv('../data/output/ar6_calibration_ebm3_rcp.csv')